# 環境初期化
はじめに実行してください。

コマンドのインストール、リポジトリのcloneを行います。

In [ ]:
!apt-get update && apt-get install -y apksigner zipalign

!mkdir /opt/apktool
!wget -q -O /opt/apktool/apktool https://raw.githubusercontent.com/iBotPeaches/Apktool/master/scripts/linux/apktool
!wget -q -O /opt/apktool/apktool.jar https://bitbucket.org/iBotPeaches/apktool/downloads/apktool_2.9.3.jar
!chmod u+x /opt/apktool/apktool

!rm -rf /content/repo
!git clone https://github.com/kairi003/NarouReaderMod.git /content/repo

import os
%env PATH=/opt/apktool:{os.environ['PATH']}
%cd /content/repo

# 署名用キーストア作成

apk署名用のキーストア `.keystore` を作成し、ダウンロードします。
既に作成済みの場合は必要ありません。

**異なるキーストアで署名されたアプリ間ではアップデートインストールができないためご注意ください。**

実行するとキーストアに設定するパスワードの入力が求められます。
パスワードは6文字以上必要です。

In [ ]:
from getpass import getpass
from google.colab import files

key_pass = getpass('keystore password: ')

!rm /tmp/.keystore
!keytool -genkeypair -v -keystore "/tmp/.keystore" -alias "narou-mod" \
  -keyalg RSA -keysize 2048 -validity 100000 \
  -dname "CN=Unknown,OU=Unknown,O=Unknown,L=Unknown,S=Unknown,C=Unknown" \
  -storepass {key_pass}

files.download('/tmp/.keystore')

# パッチ適用

1. 署名用キーストア(`.keystore`)をアップロード
1. キーストアのパスワードを入力
  - パスワードが間違っている場合ココでエラーになります
1. なろうリーダ v1.35.6 の apkファイル をアップロード
1. パッチ適用後、 `narou-mod.apk` がダウンロードされます

In [ ]:
%cd /content/repo

import subprocess
from getpass import getpass
from google.colab import files

print('署名用キーストアをアップロード')
files.upload_file('/content/repo/.keystore')
key_pass = getpass('keystore password: ')
proc = subprocess.run(['keytool', '-list', '-keystore', '.keystore'], input=key_pass.encode())
assert proc.returncode == 0, 'Incorrect Password'

print('なろうリーダ v1.35.6 の apkファイル をアップロード')
files.upload_file('/content/repo/original.apk')

%env KEY_PASS=pass:{key_pass}
!bash ./run.sh

files.download('/content/repo/narou-mod.apk')